### Get the Ground Truth Distributions
First, we will get the ground truth distributions for the patches images and their aggregated images.

In [4]:
from src.segmentation.evaluation.detectron2_evaluator import Detectron2Evaluator
import os

# varaiables
dataset_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44"
images_dir_path = os.path.join(dataset_path, "v0.1")
coco_annotations_path = os.path.join(dataset_path, "annotations", "export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1.json")
yolo_annotations_dir_path = os.path.join(dataset_path, "annotations", "yolo", "labels", "export_coco-instance_etaylor_cannabis_patches_test_26-04-2024_15-44-44_v0.1")

# get evalutor and get the ground truth boxes
detectron2_evaluator = Detectron2Evaluator(num_classes=3,coco_annotations_file_path=coco_annotations_path)
dataset_gt_boxes = detectron2_evaluator.get_annotations_for_dataset(images_directory=images_dir_path)

In [6]:
def calculate_trichome_distribution(annotations_dict):
    distribution_dict = {}
    image_distribution_dict = {}

    for image, patches in annotations_dict.items():
        image_distribution = {0: 0, 1: 0, 2: 0}
        distribution_dict[image] = {}

        for patch, annotations in patches.items():
            patch_distribution = {0: 0, 1: 0, 2: 0}

            for annotation in annotations:
                class_id = annotation['class_id']
                patch_distribution[class_id] += 1
                image_distribution[class_id] += 1

            distribution_dict[image][patch] = patch_distribution

        image_distribution_dict[image] = image_distribution

    return distribution_dict, image_distribution_dict

In [7]:
patch_distribution, image_distribution = calculate_trichome_distribution(dataset_gt_boxes)

In [8]:
# Print results
print("Patch Distribution:")
for image, patches in patch_distribution.items():
    print(f"\n{image}:")
    for patch, dist in patches.items():
        print(f"  {patch}: {dist}")

print("\nImage Distribution:")
for image, dist in image_distribution.items():
    print(f"{image}: {dist}")

Patch Distribution:

IMG_0058:
  IMG_0058_p0.png: {0: 6, 1: 2, 2: 0}
  IMG_0058_p1.png: {0: 16, 1: 2, 2: 0}
  IMG_0058_p2.png: {0: 14, 1: 7, 2: 0}
  IMG_0058_p3.png: {0: 7, 1: 6, 2: 0}
  IMG_0058_p4.png: {0: 3, 1: 1, 2: 0}
  IMG_0058_p5.png: {0: 3, 1: 1, 2: 0}
  IMG_0058_p6.png: {0: 9, 1: 4, 2: 0}
  IMG_0058_p7.png: {0: 12, 1: 5, 2: 0}
  IMG_0058_p8.png: {0: 7, 1: 3, 2: 0}

IMG_0542:
  IMG_0542_p0.png: {0: 3, 1: 0, 2: 0}
  IMG_0542_p10.png: {0: 1, 1: 7, 2: 0}
  IMG_0542_p12.png: {0: 3, 1: 10, 2: 0}
  IMG_0542_p2.png: {0: 1, 1: 5, 2: 0}
  IMG_0542_p3.png: {0: 2, 1: 5, 2: 0}
  IMG_0542_p5.png: {0: 3, 1: 13, 2: 0}
  IMG_0542_p6.png: {0: 11, 1: 7, 2: 0}
  IMG_0542_p7.png: {0: 7, 1: 0, 2: 0}
  IMG_0542_p8.png: {0: 5, 1: 1, 2: 0}

IMG_2285:
  IMG_2285_p0.png: {0: 2, 1: 5, 2: 1}
  IMG_2285_p10.png: {0: 0, 1: 6, 2: 1}
  IMG_2285_p11.png: {0: 1, 1: 12, 2: 1}
  IMG_2285_p12.png: {0: 6, 1: 19, 2: 0}
  IMG_2285_p13.png: {0: 5, 1: 11, 2: 0}
  IMG_2285_p1.png: {0: 3, 1: 7, 2: 2}
  IMG_2285_p2.png: {

In [11]:
import pandas as pd
# build dataframe from the dicts distributions
images_annotations = []

for image_number, image_dist in image_distribution.items():
    clear_count = image_dist.get(0, 0)
    cloudy_count = image_dist.get(1, 0)
    amber_count = image_dist.get(2, 0)
    total_count = clear_count + cloudy_count + amber_count

    # Calculate normalized distribution
    if total_count > 0:
        clear_normalized = clear_count / total_count
        cloudy_normalized = cloudy_count / total_count
        amber_normalized = amber_count / total_count
    else:
        clear_normalized = cloudy_normalized = amber_normalized = 0

    images_annotations.append({
        "image_number": image_number,
        "clear": clear_count,
        "cloudy": cloudy_count,
        "amber": amber_count,
        "clear_normalized": clear_normalized,
        "cloudy_normalized": cloudy_normalized,
        "amber_normalized": amber_normalized
    })
    
images_gt_df = pd.DataFrame(images_annotations)
images_gt_df

,image_number,clear,cloudy,amber,clear_normalized,cloudy_normalized,amber_normalized
0,IMG_0058,77,31,0,0.712963,0.287037,0.000000
1,IMG_0542,36,48,0,0.428571,0.571429,0.000000
2,IMG_2285,29,127,12,0.172619,0.755952,0.071429
3,IMG_1096,22,109,39,0.129412,0.641176,0.229412
4,IMG_1753,73,21,0,0.776596,0.223404,0.000000
5,IMG_2271,23,57,10,0.255556,0.633333,0.111111
6,IMG_2198,48,148,10,0.233010,0.718447,0.048544
7,IMG_1079,68,79,4,0.450331,0.523179,0.026490
8,IMG_0019,140,99,3,0.578512,0.409091,0.012397
9,IMG_1093,33,143,36,0.155660,0.674528,0.169811


In [12]:
# create patches ground truth dataframe
patches_annotations = []

for image_number, patches in patch_distribution.items():
    for patch_number, patch_dist in patches.items():
        clear_count = patch_dist.get(0, 0)
        cloudy_count = patch_dist.get(1, 0)
        amber_count = patch_dist.get(2, 0)
        total_count = clear_count + cloudy_count + amber_count

        # Calculate normalized distribution
        if total_count > 0:
            clear_normalized = clear_count / total_count
            cloudy_normalized = cloudy_count / total_count
            amber_normalized = amber_count / total_count
        else:
            clear_normalized = cloudy_normalized = amber_normalized = 0

        patches_annotations.append({
            "image_number": image_number,
            "patch_number": patch_number,
            "clear": clear_count,
            "cloudy": cloudy_count,
            "amber": amber_count,
            "clear_normalized": clear_normalized,
            "cloudy_normalized": cloudy_normalized,
            "amber_normalized": amber_normalized
        })
    
patches_gt_df = pd.DataFrame(patches_annotations)
patches_gt_df.head()

,image_number,patch_number,clear,cloudy,amber,clear_normalized,cloudy_normalized,amber_normalized
0,IMG_0058,IMG_0058_p0.png,6,2,0,0.750000,0.250000,0.0
1,IMG_0058,IMG_0058_p1.png,16,2,0,0.888889,0.111111,0.0
2,IMG_0058,IMG_0058_p2.png,14,7,0,0.666667,0.333333,0.0
3,IMG_0058,IMG_0058_p3.png,7,6,0,0.538462,0.461538,0.0
4,IMG_0058,IMG_0058_p4.png,3,1,0,0.750000,0.250000,0.0


### Prediction Models Trichomes Distributions
Here we will use the detection results from the previous experiment done for comparing detections models.
We will extract the bboxes from the detections of the models and use them to extract trichomes images for evaluation.
After extracting the trichomes images we will classify them using the Alexnet model and evalute the results.

In [ ]:
# laod the pred boxes of each model
import json
import os
import numpy as np

# load the ultralytics models pred boxes
ultralytics_models_pred_boxes_saveing_path = os.path.join("/home/etaylor/code_projects/thesis/data/models_scores", "ultralytics_models_pred_boxes_result_22_05_2024.json")
detectron2_models_pred_boxes_saveing_path = os.path.join("/home/etaylor/code_projects/thesis/data/models_scores", "detectron2_models_pred_boxes_result_22_05_2024.json")

with open(ultralytics_models_pred_boxes_saveing_path, "r") as f:
    ultralytics_models_pred_boxes = json.load(f)
    
with open(detectron2_models_pred_boxes_saveing_path, "r") as f:
    detectron2_models_pred_boxes = json.load(f)

In [22]:
import os
import cv2

def create_trichome_dataset_from_detections(detection_dict, images_directory, output_directory, extend_percentage=0):
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Process each image and crop trichomes
    for image_number, patches_detections in detection_dict.items():
        for patch_file_name, detections in patches_detections.items():
            # Extract patch number from the filename
            patch_number = patch_file_name.split('_p')[1].split('.')[0]
            
            image_path = os.path.join(images_directory, patch_file_name)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Image {image_path} not found or unable to read.")
                continue

            for i, detection in enumerate(detections):
                bbox = detection['bbox']

                x_min, y_min, x_max, y_max = map(int, bbox)

                # Calculate the width and height of the bounding box
                width = x_max - x_min
                height = y_max - y_min

                # Calculate the extension for width and height
                width_extension = int(width * extend_percentage)
                height_extension = int(height * extend_percentage)

                # Extend the bounding box coordinates
                x_min = max(x_min - width_extension, 0)
                y_min = max(y_min - height_extension, 0)
                x_max = min(x_max + width_extension, image.shape[1])
                y_max = min(y_max + height_extension, image.shape[0])

                # Crop the trichome with extended bounding box
                trichome_crop = image[y_min:y_max, x_min:x_max]

                # Save the cropped image with the desired naming convention including patch number
                output_path = os.path.join(output_directory, f"{image_number}_p{patch_number}_trichome_{i}.png")
                cv2.imwrite(output_path, trichome_crop)

                print(f"Cropped trichome saved at: {output_path}")

    print(f"Dataset creation completed. Images saved in {output_directory}")

In [ ]:
detectron2_models_pred_boxes['faster_rcnn_R_50_DC5_1x']

In [23]:
# creating the trichomes dataset of the faster_rcnn_R_50_DC5_1x mode
saving_path = "/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x"

create_trichome_dataset_from_detections(detection_dict=detectron2_models_pred_boxes['faster_rcnn_R_50_DC5_1x'],
                                        images_directory=images_dir_path,
                                        output_directory=saving_path,
                                        extend_percentage=0.1)

Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_0.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_1.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_2.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096_p10_trichome_3.png
Cropped trichome saved at: /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/model_preds_trichomes_datasets/faster_rcnn_R_50_DC5_1x/IMG_1096

### Train Alexnet Model
Now we will fine tune the alexnet model that we will use for the classification task

In [ ]:
from fastai.vision.all import *
from fastai.vision import *

data = {
        'train': '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train_26-04-2024_15-44-44/trichome_dataset_01',
        'test': '/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test_26-04-2024_15-44-44/trichome_dataset_01',
    }

# define train metrics
precision_macro_fastai = Precision(average='macro')
recall_macro_fastai = Recall(average='macro')
roc_auc_fastai = RocAuc()


# transformation and image space conversion
def custom_transform(size):
    return Resize(size, method='pad', pad_mode='zeros')

class RGB2HSV(Transform):
    def encodes(self, img: PILImage): 
        return rgb2hsv(img)
    
    
global_item_tfms=custom_transform(size=128),  # Resize and HSV transform
global_batch_tfms=[
    RGB2HSV(),
    *aug_transforms(size=128, flip_vert=True, max_rotate=10),
    Brightness(max_lighting=0.2, p=0.75),
    Contrast(max_lighting=0.2, p=0.75),
]

dls = ImageDataLoaders.from_folder(
        path=data['train'],
        item_tfms=global_item_tfms,
        batch_tfms=global_batch_tfms,
        bs=16,
        valid_pct=0.25
    )

model = vision_learner(
            dls=dls,
            arch=models.alexnet,
            metrics=[error_rate, precision_macro_fastai, recall_macro_fastai, roc_auc_fastai]
        )

model.fine_tune(epochs=50)

In [ ]:

from collections import defaultdict

# Function to predict a single image
def predict_image(img_path, model):
    img = PILImage.create(img_path)
    pred_class, _, _ = model.predict(img)
    return pred_class

# Function to aggregate results for each patch
def aggregate_patch_results(predictions):
    return {cls: predictions.count(cls) for cls in set(predictions)}

# Dictionary to store predictions for each patch
patch_predictions = defaultdict(list)

# Predict each image and store results
for img_name in os.listdir(saving_path):
    if img_name.endswith('.png'):
        img_path = os.path.join(saving_path, img_name)
        pred_class = predict_image(img_path, model)
        
        # Extract patch name from image name
        patch_name = '_'.join(img_name.split('_')[:3])  # Assumes format like IMG_0019_p10_trichome_5.png
        
        patch_predictions[patch_name].append(str(pred_class))

# Aggregate results for each patch
aggregated_results = {f"{patch}.png": aggregate_patch_results(preds) for patch, preds in patch_predictions.items()}

# Convert results to a DataFrame for easy viewing and saving
results_df = pd.DataFrame.from_dict(aggregated_results, orient='index')
results_df = results_df.fillna(0).astype(int)  # Replace NaN with 0 and convert to int


In [50]:
print(f"Length of results: {len(results_df)}")
# remove patch_number column from df

results_df.head()

Length of results: 108


,amber,clear,cloudy
IMG_1096_p10.png,8,8,19
IMG_1096_p11.png,5,3,12
IMG_1096_p1.png,3,0,2
IMG_1096_p12.png,2,4,17
IMG_1096_p3.png,1,5,0


In [55]:
import pandas as pd
import numpy as np

def normalize_patch_scores(df):
    # Create a copy of the original DataFrame
    result_df = df.copy()
    
    # Calculate normalized scores
    normalized_scores = df.div(df.sum(axis=1), axis=0)
    
    # Add normalized scores to the result DataFrame with a 'norm_' prefix
    for column in normalized_scores.columns:
        result_df[f'{column}_norm'] = normalized_scores[column]
    
    return result_df

def aggregate_image_scores(df):
    # Extract image names from index
    df['image'] = df.index.str.split('_p').str[0]
    
    # Separate original and normalized columns
    original_columns = [col for col in df.columns if not col.startswith('norm_') and col != 'image']
    norm_columns = [col for col in df.columns if col.startswith('norm_')]
    
    # Group by image and sum the scores
    image_scores = df.groupby('image')[original_columns + norm_columns].sum()
    
    # Normalize the aggregated original scores
    for column in original_columns:
        image_scores[f'agg_norm_{column}'] = image_scores[column] / image_scores[original_columns].sum(axis=1)
    
    return image_scores

# Assuming you have your results DataFrame loaded
# results_df = pd.read_csv('aggregated_results.csv', index_col=0)

# Normalize patch scores and add to the original DataFrame
patches_pred_df = normalize_patch_scores(results_df)

# Aggregate and normalize image scores
images_pred_df = aggregate_image_scores(patches_pred_df)

# add the patch_nmebr col to be first on the df
patches_pred_df['patch_number'] = patches_pred_df.index
patches_pred_df.reset_index(drop=True, inplace=True)


In [56]:
patches_pred_df.head()

,amber,clear,cloudy,amber_norm,clear_norm,cloudy_norm,image,patch_number
0,8,8,19,0.228571,0.228571,0.542857,IMG_1096,IMG_1096_p10.png
1,5,3,12,0.250000,0.150000,0.600000,IMG_1096,IMG_1096_p11.png
2,3,0,2,0.600000,0.000000,0.400000,IMG_1096,IMG_1096_p1.png
3,2,4,17,0.086957,0.173913,0.739130,IMG_1096,IMG_1096_p12.png
4,1,5,0,0.166667,0.833333,0.000000,IMG_1096,IMG_1096_p3.png


### Evaluate Regression Scores

In [41]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0  # Exclude zero values to avoid division by zero
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = calculate_mape(y_true, y_pred)
    return {'RMSE': rmse, 'MSE': mse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

def evaluate_patch_predictions(patches_gt_df, results_with_norm, model_name):
    results = {"model_name": model_name}
    trichome_types = ['clear', 'cloudy', 'amber']

    # Prepare ground truth data
    patches_gt_df['patch_id'] = patches_gt_df['image_number'] + '_' + patches_gt_df['patch_number'].str.split('.').str[0]
    patches_gt_df = patches_gt_df.set_index('patch_id')

    # Prepare prediction data
    results_with_norm = results_with_norm.reset_index().rename(columns={'index': 'patch_id'})
    results_with_norm['image_number'] = results_with_norm['patch_id'].str.rsplit('_p', 1).str[0]
    results_with_norm = results_with_norm.set_index('patch_id')

    # Merge patch-level data
    merged_patches = pd.merge(patches_gt_df, results_with_norm, 
                              left_index=True, right_index=True, 
                              suffixes=('_gt', '_pred'))
    
    for score_type in ['', 'norm_']:
        all_actuals = []
        all_preds = []
        
        for trichome_type in trichome_types:
            if score_type == '':
                actual = merged_patches[f'{trichome_type}_gt']
                pred = merged_patches[f'{trichome_type}_pred']
            else:
                actual = merged_patches[f'{trichome_type}_normalized']
                pred = merged_patches[f'{score_type}{trichome_type}']

            all_actuals.append(actual)
            all_preds.append(pred)

            metrics = calculate_metrics(actual, pred)
            for metric, value in metrics.items():
                results[f"Patch {score_type}{trichome_type} {metric}"] = value

        # Calculate overall metrics for this score type
        all_actuals = np.concatenate(all_actuals)
        all_preds = np.concatenate(all_preds)
        overall_metrics = calculate_metrics(all_actuals, all_preds)
        for metric, value in overall_metrics.items():
            results[f"Patch {score_type}Overall {metric}"] = value

    return pd.DataFrame([results])

def evaluate_aggregated_predictions(ground_truth_images_df, pred_images_df, model_name):
    results = {"model_name": model_name}
    trichome_types = ['clear', 'cloudy', 'amber']

    # Ensure the indexes match
    ground_truth_images_df = ground_truth_images_df.set_index('image_number')
    pred_images_df = pred_images_df.set_index('image')

    for score_type in ['', 'norm_']:
        all_actuals = []
        all_preds = []
        
        for trichome_type in trichome_types:
            if score_type == '':
                actual = ground_truth_images_df[trichome_type]
                pred = pred_images_df[trichome_type]
            else:
                actual = ground_truth_images_df[f'{trichome_type}_normalized']
                pred = pred_images_df[f'{score_type}{trichome_type}']

            all_actuals.append(actual)
            all_preds.append(pred)

            metrics = calculate_metrics(actual, pred)
            for metric, value in metrics.items():
                results[f"Image {score_type}{trichome_type} {metric}"] = value

        # Calculate overall metrics for this score type
        all_actuals = np.concatenate(all_actuals)
        all_preds = np.concatenate(all_preds)
        overall_metrics = calculate_metrics(all_actuals, all_preds)
        for metric, value in overall_metrics.items():
            results[f"Image {score_type}Overall {metric}"] = value

    return pd.DataFrame([results])

In [39]:
patches_pred_df.head()

,amber,clear,cloudy,norm_amber,norm_clear,norm_cloudy,image
IMG_1096_p10,8,8,19,0.228571,0.228571,0.542857,IMG_1096
IMG_1096_p11,5,3,12,0.250000,0.150000,0.600000,IMG_1096
IMG_1096_p1,3,0,2,0.600000,0.000000,0.400000,IMG_1096
IMG_1096_p12,2,4,17,0.086957,0.173913,0.739130,IMG_1096
IMG_1096_p3,1,5,0,0.166667,0.833333,0.000000,IMG_1096


In [40]:
patches_gt_df.head()

,image_number,patch_number,clear,cloudy,amber,clear_normalized,cloudy_normalized,amber_normalized
0,IMG_0058,IMG_0058_p0.png,6,2,0,0.750000,0.250000,0.0
1,IMG_0058,IMG_0058_p1.png,16,2,0,0.888889,0.111111,0.0
2,IMG_0058,IMG_0058_p2.png,14,7,0,0.666667,0.333333,0.0
3,IMG_0058,IMG_0058_p3.png,7,6,0,0.538462,0.461538,0.0
4,IMG_0058,IMG_0058_p4.png,3,1,0,0.750000,0.250000,0.0


In [ ]:
patches_results = evaluate_patch_predictions(patches_gt_df, patches_pred_df, model_name='Faster R-CNN and AlexNet')

In [ ]:
# TODO: organize the predictions dataframes that it would be easier to compare the results
# TODO: align the format of the pred and gt dfs
# TODO: calc evaluation metrics
# TODO: add comparison for the other detection models and eval results